In [ ]:
from orion.data import load_signal, load_anomalies

# 1. Data

In [ ]:
signal_name = 'multivariate/S-1'

data = load_signal(signal_name)
data = data[8000:10000]

data.head()

# 2. Pipeline

In [ ]:
from mlblocks import MLPipeline

pipeline_name = 'chronos2'

pipeline = MLPipeline(pipeline_name)
    

### Hyperparameters

The Chronos2 pipeline can be customized with the following hyperparameters:

| Primitive | Parameter | Default | Description |
|-----------|-----------|---------|-------------|
| time_segments_aggregate | `interval` | 21600 | Aggregation interval in seconds |
| time_segments_aggregate | `method` | "mean" | Aggregation method (mean, median, sum) |
| rolling_window_sequences | `window_size` | 250 | Context window size |
| **Chronos2** | `pred_len` | 1 | Prediction horizon length |
| **Chronos2** | `batch_size` | 32 | Batch size for inference |
| **Chronos2** | `target` | 0 | Target column index (multivariate) |
| find_anomalies | `window_size_portion` | 0.33 | Portion of data for window |
| find_anomalies | `fixed_threshold` | True | Use fixed vs dynamic threshold |

In [ ]:
hyperparameters = {
    "mlstars.custom.timeseries_preprocessing.time_segments_aggregate#1": {
        "time_column": "timestamp",
        "interval": 21600,        
        "method": "mean"        
    },
    
    "mlstars.custom.timeseries_preprocessing.rolling_window_sequences#1": {
        "target_column": 0,
        "window_size": 250
    },
    
    "orion.primitives.chronos2.Chronos2#1": {
        "pred_len": 1,          
        "batch_size": 32,       
        "target": 0,            
    },
    
    "orion.primitives.timeseries_anomalies.find_anomalies#1": {
        "window_size_portion": 0.33,
        "window_step_size_portion": 0.1,
        "fixed_threshold": True
    }
}

pipeline.set_hyperparameters(hyperparameters)

## step by step execution

MLPipelines are compose of a squence of primitives, these primitives apply tranformation and calculation operations to the data and updates the variables within the pipeline. To view the primitives used by the pipeline, we access its `primtivies` attribute. 

The `UniTS` contains 6 primitives. we will observe how the `context` (which are the variables held within the pipeline) are updated after the execution of each primitive.

In [ ]:
pipeline.primitives

### time segments aggregate
this primitive creates an equi-spaced time series by aggregating values over fixed specified interval.

* **input**: `X` which is an n-dimensional sequence of values.
* **output**:
    - `X` sequence of aggregated values, one column for each aggregation method.
    - `index` sequence of index values (first index of each aggregated segment).

In [ ]:
context = pipeline.fit(data, output_=0)
context.keys()

In [ ]:
for i, x in list(zip(context['index'], context['X']))[:5]:
    print("entry at {} has value {}".format(i, x))

### SimpleImputer
this primitive is an imputation transformer for filling missing values.
* **input**: `X` which is an n-dimensional sequence of values.
* **output**: `X` which is a transformed version of X.

In [ ]:
step = 1

context = pipeline.fit(**context, output_=step, start_=step)
context.keys()

### rolling window sequence
this primitive generates many sub-sequences of the original sequence. it uses a rolling window approach to create the sub-sequences out of time series data.

* **input**: 
    - `X` n-dimensional sequence to iterate over.
    - `index` array containing the index values of X.
* **output**:
    - `X` input sequences.
    - `y` target sequences.
    - `index` first index value of each input sequence.
    - `target_index` first index value of each target sequence.

In [ ]:
step = 2

context = pipeline.fit(**context, output_=step, start_=step)
context.keys()

In [ ]:
# after slicing X into multiple sub-sequences
# we obtain a 3 dimensional matrix X where
# the shape indicates (# slices, window size, 1)
# and similarly y is (# slices, target size)

print("X shape = {}\ny shape = {}\nindex shape = {}\ntarget index shape = {}".format(
    context['X'].shape, context['y'].shape, context['index'].shape, context['target_index'].shape))

### Chronos-2
This is the forecasting step using Amazon Chronos-2 Time Series Foundation Model. You can read more about it in the [related paper](https://arxiv.org/abs/2510.15821). The [Huggingface Repo](https://huggingface.co/amazon/chronos-2) has additional helpful information about the use of the model. This is a multivariate model that does single target predictions.

* **input**: 
    - `X` n-dimensional array containing the input sequences for the model.
* **output**: 
    - `y_hat` predicted values for target column

In [ ]:
step = 3

context = pipeline.fit(**context, output_=step, start_=step)
context.keys()

In [ ]:
context['y_hat'].shape

### regression errors

this primitive computes an array of errors comparing predictions and expected output.

* **input**: 
    - `y` ground truth.
    - `y_hat` forecasted values.
* **output**: `errors` array of errors.

In [ ]:
step = 4

context = pipeline.fit(**context, output_=step, start_=step)
context.keys()

### find anomalies

this primitive finds anomalies from sequence of errors

* **input**: 
    - `errors` array of errors
    - `target_index` indices
* **output**: `anomalies`.

In [ ]:
step = 5

context = pipeline.fit(**context, output_=step, start_=step)
context.keys()

In [ ]:
context['anomalies']

## 3. Evaluate performance

In this next step we will load some already known anomalous intervals and evaluate how
good our anomaly detection was by comparing those with our detected intervals.

For this, we will first load the known anomalies for the signal that we are using:

In [ ]:
from orion.data import load_anomalies

ground_truth = load_anomalies('S-1')
ground_truth

In [ ]:
anomalies = []
for ano in context['anomalies']:
    anomalies.append((ano[0], ano[1]))
anomalies

In [ ]:
from orion.evaluation import contextual_confusion_matrix, contextual_f1_score

start, end = context['index'][0], context['index'][-1]

contextual_confusion_matrix(ground_truth, anomalies, start = start, end = end, weighted=False)

In [ ]:
contextual_f1_score(ground_truth, anomalies, start = start, end = end, weighted=False)